In [81]:
import pandas as pd
import datetime

In [82]:
#de la carpeta Data estraermos el .csv que contine la data ya limpia.
Data=pd.read_csv("../Data/Data_Limpia.csv")
Data.head()

,CaptureDate,Aerolinea,Fecha,Departure,Arrive,Hora_Salida,Hora_LLegada,Price,Tipo
0,2023-04-18,Latam,2023-04-23,SMR,MDE,20:35:00,21:51:00,993760,Directo
1,2023-04-18,Latam,2023-04-23,SMR,MDE,07:18:00,10:44:00,214790,1_parada
2,2023-04-18,Latam,2023-04-23,SMR,MDE,07:18:00,11:11:00,214790,1_parada
3,2023-04-18,Latam,2023-04-23,SMR,MDE,07:18:00,12:56:00,214790,1_parada
4,2023-04-18,Latam,2023-04-23,SMR,MDE,07:43:00,11:11:00,227280,1_parada


In [83]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7055 entries, 0 to 7054
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CaptureDate   7055 non-null   object
 1   Aerolinea     7055 non-null   object
 2   Fecha         7055 non-null   object
 3   Departure     7055 non-null   object
 4   Arrive        7055 non-null   object
 5   Hora_Salida   7055 non-null   object
 6   Hora_LLegada  7055 non-null   object
 7   Price         7055 non-null   int64 
 8   Tipo          7055 non-null   object
dtypes: int64(1), object(8)
memory usage: 496.2+ KB


In [84]:
#Transformar la data a los type necesarios para la creacion de las nuevas variables
Data['CaptureDate']=pd.to_datetime(Data['CaptureDate'])
Data['Fecha']=pd.to_datetime(Data['Fecha'])

Data['Hora_Salida'] = pd.to_datetime(Data['Hora_Salida'], format='%H:%M:%S').dt.time
Data['Hora_LLegada'] = pd.to_datetime(Data['Hora_LLegada'], format='%H:%M:%S').dt.time

#### Creacion de las nuevas variables
derivadas de las variables principales

- Fechas: Se deriva como Departure_Day, Departure_Month, Departure_Year, Weekday , Season, Holidays
- Hora_Salida: en Departure_time
- DepartureDate y Fecha: su difenrecia crea la variable Consultation_period

In [85]:
# Extraer las columnas requeridas
Data['Departure_Day'] = Data['Fecha'].dt.day
Data['Departure_Month'] = Data['Fecha'].dt.month
Data['Departure_Year'] = Data['Fecha'].dt.year
Data['Weekday'] = Data['Fecha'].dt.weekday


In [86]:
# Definir una función que devuelva "día" o "noche" en función de la hora
def calcular_horario(hora):
    """
    This function takes a time input and returns a string indicating whether it is in the morning,
    afternoon, evening, or late night.
    
    :param hora: The input parameter "hora" is expected to be a time object in the format of "HH:MM:SS"
    (hours, minutes, seconds) or a string that can be parsed into a time object
    :return: a string indicating the time of day based on the input time. The possible return values are
    "Madrugada" (dawn), "Mañana" (morning), "Tarde" (afternoon), or "Noche" (night).
    """
    hora_objeto = hora 
    if hora_objeto < datetime.time(6,0,0):
        return "Madrugada"
    elif hora_objeto < datetime.time(12,0,0):
        return "Mañana"
    elif hora_objeto < datetime.time(18,0,0):
        return "Tarde"
    else:
        return "Noche"

# Crear una nueva columna "momento_del_dia" que indique si la observación se realizó durante el día o la noche
Data['Departure_time'] = Data['Hora_Salida'].apply(lambda x : calcular_horario(x))

In [87]:
from datetime import datetime
# Definimos las fechas comunes para vacacionar en Colombia
vacaciones = [('12-15', '01-15'), ('06-20', '07-20'), ('03-21', '04-07'), ('05-01', '05-03')]

# Función para verificar si una fecha está dentro de las fechas comunes para vacacionar
def esta_en_vacaciones(fecha):
    for inicio, fin in vacaciones:
        fecha_inicio = datetime.strptime(f'{fecha.year}-{inicio}', '%Y-%m-%d').date()
        fecha_fin = datetime.strptime(f'{fecha.year}-{fin}', '%Y-%m-%d').date()
        if fecha_inicio <= fecha.date() <= fecha_fin:
            return True
    return False

# Creamos la columna indicando si la fecha está en las fechas comunes para vacacionar
Data['Holidays'] = Data['Fecha'].apply(lambda x: esta_en_vacaciones(x))

In [88]:
Data

,CaptureDate,Aerolinea,Fecha,Departure,Arrive,Hora_Salida,Hora_LLegada,Price,Tipo,Departure_Day,Departure_Month,Departure_Year,Weekday,Departure_time,Holidays
0,2023-04-18,Latam,2023-04-23,SMR,MDE,20:35:00,21:51:00,993760,Directo,23,4,2023,6,Noche,False
1,2023-04-18,Latam,2023-04-23,SMR,MDE,07:18:00,10:44:00,214790,1_parada,23,4,2023,6,Mañana,False
2,2023-04-18,Latam,2023-04-23,SMR,MDE,07:18:00,11:11:00,214790,1_parada,23,4,2023,6,Mañana,False
3,2023-04-18,Latam,2023-04-23,SMR,MDE,07:18:00,12:56:00,214790,1_parada,23,4,2023,6,Mañana,False
4,2023-04-18,Latam,2023-04-23,SMR,MDE,07:43:00,11:11:00,227280,1_parada,23,4,2023,6,Mañana,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7050,2023-04-17,Latam,2023-05-24,MDE,BOG,17:27:00,20:57:00,278020,1_parada,24,5,2023,2,Tarde,False
7051,2023-04-17,Latam,2023-05-24,MDE,BOG,17:27:00,22:13:00,278020,1_parada,24,5,2023,2,Tarde,False
7052,2023-04-17,Latam,2023-05-24,MDE,BOG,17:27:00,22:46:00,278020,1_parada,24,5,2023,2,Tarde,False
7053,2023-04-17,Latam,2023-05-24,MDE,BOG,18:50:00,23:00:00,373450,1_parada,24,5,2023,2,Noche,False


In [90]:
Data[Data['Holidays']==True]

,CaptureDate,Aerolinea,Fecha,Departure,Arrive,Hora_Salida,Hora_LLegada,Price,Tipo,Departure_Day,Departure_Month,Departure_Year,Weekday,Departure_time,Holidays
139,2023-04-18,Latam,2023-05-01,BOG,MDE,05:11:00,06:10:00,257560,Directo,1,5,2023,0,Madrugada,True
140,2023-04-18,Latam,2023-05-01,BOG,MDE,06:00:00,06:59:00,257560,Directo,1,5,2023,0,Mañana,True
141,2023-04-18,Latam,2023-05-01,BOG,MDE,07:00:00,07:59:00,257560,Directo,1,5,2023,0,Mañana,True
142,2023-04-18,Latam,2023-05-01,BOG,MDE,07:53:00,08:52:00,257560,Directo,1,5,2023,0,Mañana,True
143,2023-04-18,Latam,2023-05-01,BOG,MDE,08:05:00,09:04:00,257560,Directo,1,5,2023,0,Mañana,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6855,2023-04-17,Latam,2023-05-02,MDE,BOG,17:21:00,21:19:00,424620,1_parada,2,5,2023,1,Tarde,True
6856,2023-04-17,Latam,2023-05-02,MDE,BOG,17:21:00,23:05:00,424620,1_parada,2,5,2023,1,Tarde,True
6857,2023-04-17,Latam,2023-05-02,MDE,BOG,20:17:00,23:00:00,372500,1_parada,2,5,2023,1,Noche,True
6858,2023-04-17,Latam,2023-05-02,MDE,BOG,22:23:00,05:15:00,297820,1_parada,2,5,2023,1,Noche,True
